## Start programme

In [1]:
# widgets
from ipywidgets import interact, interactive
import ipywidgets as widgets
from IPython.display import display, Markdown

# YT scrapper
!apt update
!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import files
import pandas as pd
import numpy as np

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [824 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Youtube comment scrapper

In [ ]:
#@title YouTube video link
display(Markdown("paste the URL of the video to scrap the comments"))

yt_link_input_field = widgets.Text(
    placeholder = 'YouTube video url',
    description='',
    disabled=False,
)
display(yt_link_input_field)

paste the URL of the video to scrap the comments

Text(value='', placeholder='YouTube video url')

In [ ]:
#@title Collect comments (can take some time, even few minutes)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)

############## Youtube link widget
yt_link = yt_link_input_field.value
comments = []

wait = WebDriverWait(wd,15)
wd.get(yt_link)

for item in range(100): 
  wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
  time.sleep(2)

for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
  comments.append(comment.text)

# prepare pandas dataframe to be saved
comments_df = pd.DataFrame(comments, columns=['comment'], )
# drop 1st 6 rows where no comments are saved
comments_df = comments_df.drop(np.arange(0,6))

In [ ]:
#@title Download collected comments in chosen format

# button onclick function
def saveFile(dummy):
  format = formatChoice.value
  if format == 'csv':
    comments_df.to_csv('comments.csv', index=False) 
  elif format == 'xlsx':
    comments_df.to_excel('comments.xlsx', index=False)
  elif format == 'txt':
    comments_df.to_csv('comments.txt', sep='\t') 
  files.download(f'comments.{format}')
  display(Markdown(f'{format} file saved'))

formatChoice =widgets.ToggleButtons(
    options=['txt', 'csv', 'xlsx'],
    description='format',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # tooltips=[],
#     icons=['check'] * 3
)
button = widgets.Button(
    description='save',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
display(formatChoice)
print('')

button.on_click(saveFile)
display(button)

ToggleButtons(description='format', options=('txt', 'csv', 'xlsx'), value='txt')

Button(description='save', icon='check', style=ButtonStyle(), tooltip='Click to download')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

xlsx file saved